In [18]:
import heapq
import numpy as np

In [20]:
def corr(arr1, arr2):
    leng = len(arr1)
    mean1 = sum(arr1) / leng
    mean2 = sum(arr2) / leng
    var1, var2, covar = 0, 0, 0

    for idx in range(leng):
        var1 += (arr1[idx] - mean1)**2
        var2 += (arr2[idx] - mean2)**2
        covar = (arr1[idx] - mean1)*(arr2[idx] - mean2)
    
    var1 = var1 / (leng-1)
    var2 = var2 / (leng-1)
    covar = covar / (leng-1)

    return covar / np.sqrt(var1*var2)

In [21]:
corr([1,2,3,4,5,6,7], [2,3,4,6,8,9,10])

0.29777500019127906